# Logistische Regression

**Lasso-** oder **Ridge-Regularisierung** hilft Overfitting zu vermeiden. Der F-beta Score kann dann verwendet werden, um die Leistung des regulierten Modells zu bewerten.
- **Lasso** (L1): Setzt einige Koeffizienten auf null, was einer Feature-Selektion entspricht.
- **Ridge** (L2): Schrumpft die Koeffizienten, ohne sie auf null zu setzen, was die Varianz reduziert. 


1. Lade den unbalancierten Datensatz.
2. Splitte den Datensatz in Trainings- und Testdaten.
3. Trainiere ein Modell mit Ridge-Regularisierung.
4. Trainiere ein Modell mit Lasso-Regularisierung.
5. Berechne den F-beta Score für beide Modelle auf den Testdaten.

In [9]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split

# Lade den Datensatz
train_data_loaded = pd.read_csv('../data/train_data_2024-07-31.csv')
X = train_data_loaded.drop(columns=['UKATEGORIE'])
y = train_data_loaded['UKATEGORIE']

# Splitte den Datensatz in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Modell mit Ridge-Regularisierung (L2)
model_ridge = LogisticRegression(penalty='l2', solver='lbfgs', C=1.0, max_iter = 150, class_weight={0: 1, 1: 9})
model_ridge.fit(X_train, y_train)
y_pred_ridge = model_ridge.predict(X_test)
fbeta_ridge = fbeta_score(y_test, y_pred_ridge, beta=2)
print(f'Ridge (L2) F-beta Score: {fbeta_ridge}', '\n')


# Extrahiere die Koeffizienten
coefficients = model_ridge.coef_

# Erstelle ein DataFrame mit den Features und ihren Koeffizienten
coeff_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': coefficients.flatten()
})

# Sortiere das DataFrame nach den absoluten Werten der Koeffizienten
coeff_df['Abs_Coefficient'] = coeff_df['Coefficient'].abs()
coeff_df = coeff_df.sort_values(by='Abs_Coefficient', ascending=False)

# Identifiziere die wichtigsten Features
important_features = coeff_df.head(5)  # z.B. die Top 5 Features
print(important_features[['Feature', 'Coefficient']], '\n')


# Modell mit Lasso-Regularisierung (L1)
model_lasso = LogisticRegression(penalty='l1', solver='liblinear', C=1.0, max_iter = 150, class_weight={0: 1, 1: 9})
model_lasso.fit(X_train, y_train)
y_pred_lasso = model_lasso.predict(X_test)
fbeta_lasso = fbeta_score(y_test, y_pred_lasso, beta=2)
print(f'Lasso (L1) F-beta Score: {fbeta_lasso}', '\n')

#Zeige die Koeffizienten der Ridge-Regularisierung
#Bei der Lasso-Regularisierung können Sie die korrelierten Features identifizieren, indem Sie die Koeffizienten des trainierten Modells analysieren. Features mit nicht-null Koeffizienten sind diejenigen, die vom Modell als relevant erachtet werden. 


# Extrahiere die nicht-null Koeffizienten
coef_series = pd.Series(model_lasso.coef_[0])
relevant_features = coef_series[coef_series != 0].index

# Erstelle ein DataFrame der relevanten Features
X_relevant = X_train.iloc[:, relevant_features]

# Berechne die Korrelationsmatrix der relevanten Features
correlation_matrix = X_relevant.corr()

# Extrahiere die nicht-null Koeffizienten
coef_series = pd.Series(model_lasso.coef_[0])
relevant_features = coef_series[coef_series != 0].index

# Anzahl der auf null gesetzten Features
num_null_features = (coef_series == 0).sum()
print(f'Lasso-Regutlierung: Anzahl der auf null gesetzten Features: {num_null_features}')

print("Korrelationsmatrix der relevanten Features:")
print(correlation_matrix)

/home/anne/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ridge (L2) F-beta Score: 0.46373416623286484 

       Feature  Coefficient
8       IstPKW    -0.390694
10     IstKrad     0.356974
9      IstFuss     0.334548
6   ULICHTVERH     0.245873
7       IstRad    -0.236600 

Lasso (L1) F-beta Score: 0.4696655548544127 

Lasso-Regutlierung: Anzahl der auf null gesetzten Features: 0
Korrelationsmatrix der relevanten Features:
                UJAHR    UMONAT   USTUNDE  UWOCHENTAG      UART     UTYP1  \
UJAHR        1.000000  0.042393  0.012306   -0.002894 -0.014854 -0.006403   
UMONAT       0.042393  1.000000  0.003593    0.013820 -0.025604  0.000857   
USTUNDE      0.012306  0.003593  1.000000    0.015724 -0.008369  0.030355   
UWOCHENTAG  -0.002894  0.013820  0.015724    1.000000  0.000182 -0.001436   
UART        -0.014854 -0.025604 -0.008369    0.000182  1.000000 -0.282726   
UTYP1       -0.006403  0.000857  0.030355   -0.001436 -0.282726  1.000000   
ULICHTVERH  -0.003904  0.071547  0.237957    0.019494  0.053933 -0.063918   
IstRad       0.

## Ridge-Regularisierung (L2)
##### Vorteile:  
Reduziert Overfitting, indem es die Größe der Koeffizienten beschränkt.
Funktioniert gut, wenn alle Features relevant sind.
Stabilisiert die Lösung, besonders bei multikollinearen Daten.
##### Nachteile:  
Kann nicht irrelevante Features vollständig eliminieren (Koeffizienten werden nur klein, aber nicht null).
Kann bei sehr vielen irrelevanten Features weniger effektiv sein.
## Lasso-Regularisierung (L1)
##### Vorteile:  
Kann irrelevante Features vollständig eliminieren (Koeffizienten werden null), was zu sparsamen Modellen führt.
Automatische Feature-Selektion, was die Interpretierbarkeit verbessert.
##### Nachteile:
Kann bei hoch korrelierten Features instabil sein (wählt zufällig eines der korrelierten Features aus).
Kann bei sehr vielen relevanten Features weniger effektiv sein, da es einige Koeffizienten auf null setzen könnte.